<a href="https://colab.research.google.com/github/Maxie503/LangChain.CampusX/blob/main/13_retrievers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WikipediaRetriever

In [2]:
from langchain_community.retrievers import WikipediaRetriever
retriever=WikipediaRetriever()
query ="Tell me about india and sri lanka"
docs_wiki = retriever.invoke(query)

In [51]:
for doc in docs_wiki:
    print(doc.page_content[:500])

SriLankan Airlines  is the flag carrier of Sri Lanka and a member airline of the Oneworld airline alliance. It was launched in 1979 as Air Lanka following the termination of operations of the original Sri Lankan flag carrier Air Ceylon. As of today, it is Sri Lanka's largest airline by number of aircraft and destinations. Its hub is Bandaranaike International Airport.
Following its partial acquisition in 1998 by Emirates, it was re-branded and the current livery was introduced. In 2008, the gove
No Fire Zone: In the Killing Fields of Sri Lanka is an investigative documentary about the final weeks of the Sri Lankan Civil War. Released by Channel 4, the documentary covers the period from September 2008 until the end of the war in 2009 in which thousands of Tamil people were killed by shelling and extrajudicial executions by the Sri Lankan Army including Balachandran Prabhakaran, the 12-year-old son of the slain Liberation Tigers of Tamil Eelam (LTTE) Chief Velupillai Prabhakaran. The Sri

# Vector store

In [4]:
from langchain.vectorstores import Chroma, FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DocugamiLoader
from langchain.schema import Document

In [5]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [6]:
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
vector_storage1 = Chroma.from_documents(documents,embedding=embed_model)

# similarity Search with Chroma VS

In [8]:
retriever_simi = vector_storage1.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [9]:
results1 = retriever_simi.invoke("what is Chroma?")

In [10]:
for index,doc in enumerate(results1):
  print(doc.page_content)

Chroma is a vector database optimized for LLM-based search.
Embeddings convert text into high-dimensional vectors.


# MMR Search with FAISS Vector Store

In [11]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain makes it fast to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [12]:
vector_storage2 = FAISS.from_documents(docs, embedding=embed_model)

# similarity

In [13]:
retriever_simi = vector_storage2.as_retriever(search_type="similarity", search_kwargs={"k": 3})
results2= retriever_simi.invoke("what is langchain?")
for i, doc in enumerate(results2):
    print("Results=",i, doc.page_content)

Results= 0 LangChain supports Chroma, FAISS, Pinecone, and more.
Results= 1 LangChain is used to build LLM based applications.
Results= 2 LangChain makes it fast to work with LLMs.


# MMR

In [14]:
retriever_mmr = vector_storage2.as_retriever(search_type="mmr", search_kwargs={"k": 3, "lambda_mult": 0.5})
results2= retriever_mmr.invoke("what is langchain?")
for i, doc in enumerate(results2):
    print("Results=",i, doc.page_content)

Results= 0 LangChain supports Chroma, FAISS, Pinecone, and more.
Results= 1 LangChain is used to build LLM based applications.
Results= 2 Embeddings are vector representations of text.


# Multiquery Retriever

In [15]:
from langchain.retrievers.multi_query import MultiQueryRetriever


In [16]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [17]:
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    max_length=100,
    do_sample=False
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cpu


In [18]:
vector_storage3 = FAISS.from_documents(all_docs, embedding=embed_model)

In [19]:
nquery=3
simi_rete = vector_storage3.as_retriever(search_type="mmr", search_kwargs={"k": nquery, "lambda_mult" : 1})
multi_retriever = multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vector_storage3.as_retriever(search_type="mmr",search_kwargs={"k": nquery, "lambda_mult" : 1}),llm=llm)

In [20]:
resultsMulti = multi_retriever.invoke("How to improve energy levels and maintain balance healthy life?")
resultsSS = simi_rete.invoke("How to improve energy levels and maintain balance healthy life?")
print("--------MultiQuery Retriever----------")
for index, mdoc in enumerate(resultsMulti):
    print("Results=",index, mdoc.page_content)

print("----------SS Retriever----------")
for index, ssdoc in enumerate(resultsSS):
    print("Results=",index, ssdoc.page_content)

--------MultiQuery Retriever----------
Results= 0 Drinking sufficient water throughout the day helps maintain metabolism and energy.
Results= 1 Consuming leafy greens and fruits helps detox the body and improve longevity.
Results= 2 Mindfulness and controlled breathing lower cortisol and improve mental clarity.
----------SS Retriever----------
Results= 0 Drinking sufficient water throughout the day helps maintain metabolism and energy.
Results= 1 Consuming leafy greens and fruits helps detox the body and improve longevity.
Results= 2 The solar energy system in modern homes helps balance electricity demand.


# ContextualCompressionRetriever

In [22]:
from langchain.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever

In [23]:
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import pipeline

hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    max_length=256,
    do_sample=False
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)
embed_model_ccr = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Device set to use cpu


In [24]:
# Document objects
docs_ccr = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [25]:
vector_storage_ccr = FAISS.from_documents(docs_ccr,embedding=embed_model_ccr)

In [37]:
base_retriever=vector_storage_ccr.as_retriever(search_kwargs={"k": 5})

In [42]:
%pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.8 MB/s eta 0:00:00


In [43]:
from langchain.retrievers.document_compressors.chain_extract import LLMChainExtractor
from langchain_groq import ChatGroq

groq_llm = ChatGroq(model_name="llama-3.3-70b-versatile", api_key="gsk_SLhReShWaJ78t62ysC1VyqUWhdyb3FYYuQItRRAb69hReQhkagugQgSmDwZ")

compressor = LLMChainExtractor.from_llm(groq_llm)

In [44]:
retriever_ccr = ContextualCompressionRetriever(base_compressor=compressor,base_retriever=base_retriever)

In [45]:
ccr_results = retriever_ccr.invoke("What is photosynthesis?")

In [46]:
for i, doc in enumerate(ccr_results):
  print('Results=',i,doc.page_content,'\n')

Results= 0 Photosynthesis is the process by which green plants convert sunlight into energy. 

Results= 1 The chlorophyll in plant cells captures sunlight during photosynthesis. 

Results= 2 Photosynthesis does not occur in animal cells. 

